In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
X = pd.read_csv('XGBdata')
Y = pd.read_csv('XGBtarget')
X.drop('Unnamed: 0', axis=1, inplace=True)
Y.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
X['20'] = X['20'].astype('str')
X['21'] = X['21'].astype('str')

In [4]:
df = pd.get_dummies(X)

In [21]:
Y

,0
0,2811.0
1,2811.0
2,2811.0
3,2811.0
4,2811.0
...,...
301905,461.0
301906,461.0
301907,485.0
301908,485.0


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size = 0.3, random_state = 42)

In [23]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,  shuffle=True, random_state=42)

In [6]:
from sklearn.metrics import mean_absolute_error as MAE

In [25]:
# xgb_r = xgb.XGBRegressor(objective ='reg:linear',
#                   n_estimators = 100, seed = 42)

In [13]:
import scipy.stats as stats
param_rand_xgb = {
    'max_depth':range(8,13,1),
    'learning_rate': stats.uniform(0.01,1),
    'subsample':stats.uniform(0.01, 0.99),
    'colsample_bytree':stats.uniform(0.01, 0.99),
    'reg_alpha':stats.uniform(0.01, 1),
    'reg_lambda':stats.uniform(0.01, 1),
    'n_estimators': [1000]
}


In [26]:
xgb = xgb.XGBRegressor(seed = 42)

In [27]:
from sklearn.model_selection import RandomizedSearchCV
rand_xgb = RandomizedSearchCV(estimator=xgb, param_distributions=param_rand_xgb, 
                              scoring='neg_mean_squared_error', n_jobs=-1,pre_dispatch='2*n_jobs', verbose=1, random_state=42, n_iter=100)

In [28]:
rand_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [ ]:
pred = rand_xgb.predict(X_test)

/home_nfs/jiangyue/anaconda3/envs/pytorch-gpu/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [ ]:
mae = MAE(y_test, pred)
print(mae)

7.248989024176736


In [ ]:
rand_xgb.best_params_

{'colsample_bytree': 0.7786488311345768,
 'learning_rate': 0.35080354025301785,
 'max_depth': 12,
 'reg_alpha': 0.24859685979034019,
 'reg_lambda': 0.6898447799002458,
 'subsample': 0.7425096728429008}

In [ ]:
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

In [11]:
xgbr = xgb.XGBRegressor(seed = 42)

In [19]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


/home_nfs/jiangyue/anaconda3/envs/pytorch-gpu/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home_nfs/jiangyue/anaconda3/envs/pytorch-gpu/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home_nfs/jiangyue/anaconda3/envs/pytorch-gpu/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/home_nfs/jiangyue/anaconda3/envs/pytorch-gpu/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  war

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, seed=42,
                                    subsample=None, tree_method=None,
                       

In [15]:
# xgb_r.fit(X_train, y_train)

In [7]:
import joblib
clf = joblib.load("XGB_10sid.pkl")

In [8]:
pred = clf.predict(X_test)

/home_nfs/jiangyue/anaconda3/envs/pytorch-gpu/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [ ]:
# pred = xgb_r.predict(X_test)

In [27]:
clf.best_params_

{'colsample_bytree': 0.3,
 'learning_rate': 0.1,
 'max_depth': 10,
 'n_estimators': 1000}

In [29]:
import joblib
joblib.dump(clf, 'XGB_10sid.pkl')
# joblib.load("model_file_name.pkl")

['XGB_10sid.pkl']

In [9]:
mae = MAE(y_test, pred)

In [10]:
mae

6.310811919237748

In [11]:
y_test = y_test.rename(columns =  {'0': 'target'})

In [12]:
abs_error = np.abs(y_test['target']-pred)

In [18]:
μ = np.mean(abs_error)
σ = np.std(abs_error)
print(f"μ = {μ:.4f}, σ = {σ:.4f}, max = {abs_error.max():.4f}")

μ = 6.3108, σ = 16.6800, max = 1257.9485
